In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint

# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# load in keys from .env. Have in format API_KEY="".
load_dotenv()
API_KEY = os.getenv("API_KEY")
LOCATION = os.getenv("LOCATION")
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456

In [2]:
from google.cloud import aiplatform

aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")

In [3]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain, PromptTemplate
from langchain.llms import vertexai
from vertexai.preview.language_models import TextGenerationModel
from langchain.agents import AgentType, initialize_agent

In [4]:
import json

from tools import GetMenuTool

## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [5]:
from square.client import Client
from square.api.catalog_api import CatalogApi
from pydantic import BaseModel, Field, ConfigDict
from langchain.tools.base import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
client = Client(
    square_version='2023-08-16',
    access_token=API_KEY,
    environment='sandbox')

In [6]:
tools = [GetMenuTool()]

In [7]:
mrkl = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_execution_time=5,
)

In [11]:
mrkl.run("What types of Danish pastry are available?")



> Entering new AgentExecutor chain...
 The get_menu_tool can provide information on Danish pastries
Action: get_menu_tool
Action Input: 
Observation: [{'type': 'Muffin', 'modifiers': ['Small', ' Blueberry'], 'item_id': 'QHIFP5QQD373ZLO5P7FHGTCM', 'price': 300}, {'type': 'Muffin', 'modifiers': ['Small', ' Banana Chocolate Chip'], 'item_id': 'JNYKOOGSGGF7XEKJBUNEK5WB', 'price': 300}, {'type': 'Muffin', 'modifiers': ['Small', ' Radish'], 'item_id': '36X25BSZPXURAJEU7KETG6IV', 'price': 300}, {'type': 'Muffin', 'modifiers': ['Medium', ' Blueberry'], 'item_id': 'CZYCQBFTH54KTQDRHQQA5PHH', 'price': 400}, {'type': 'Muffin', 'modifiers': ['Medium', ' Banana Chocolate Chip'], 'item_id': 'NIBVSCGUB6CT4VGZPBPGCHUU', 'price': 400}, {'type': 'Muffin', 'modifiers': ['Medium', ' Radish'], 'item_id': 'IN5G7QOFNW3KPGA6LACUCY2W', 'price': 400}, {'type': 'Muffin', 'modifiers': ['Large', ' Blueberry'], 'item_id': 'I7VXGGKOINY3XTX5O3BIACMD', 'price': 500}, {'type': 'Muffin', 'modifiers': ['Large', ' Banan

'Blueberry, Banana Chocolate Chip, and Radish'